In [2]:
from quadmodel.quadmodel import QuadLensSystem   
from quadmodel.deflector_models.power_law_shear import PowerLawShear
from quadmodel.macromodel import MacroLensModel
from quadmodel.data import Quad as LensedQuasar
from quadmodel.deflector_models.sis import SIS
from quadmodel.Solvers.hierachical import HierarchicalOptimization
from quadmodel.Solvers.brute import BruteOptimization
from quadmodel.util import approx_theta_E

import os
import subprocess

from pyHalo.single_realization import SingleHalo
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions

from pyHalo.preset_models import WDMLovell2020
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt

### This function generates realizations and computes flux ratios, reading them out into a file

In [1]:
def forward_model_wdm(output_path, job_index, n_realizations, zlens, zsource, x_image, y_image, magnifications, 
                  sigma_sub_prior, log_mhm_prior, alpha_prior, delta_los_prior, source_size_prior_1, source_size_prior_2,
                    log10_host_mass_prior, gamma_macro_prior, astrometric_uncertainty=0.003, verbose=False):
    
    filename_parameters = output_path + 'job_'+str(job_index) + '/parameters.txt'
    filename_mags1 = output_path + 'job_'+str(job_index) + '/fluxes_1.txt'
    filename_mags2 = output_path + 'job_'+str(job_index) + '/fluxes_2.txt'
    
    if os.path.exists(output_path + 'job_'+str(job_index)) is False:
        proc = subprocess.Popen(['mkdir', output_path + 'job_'+str(job_index)])
        proc.wait()
    
    if verbose:
        print('reading output to files: ')
        print(filename_parameters)
        print(filename_mags1)
        print(filename_mags2)
    
    R_ein_approx = approx_theta_E(x_image, y_image)
    cone_opening_angle = 6 * R_ein_approx
    
    parameter_array = None
    magnifications_1 = None
    magnifications_2 = None
    
    for i in range(0, n_realizations):
        
        sigma_sub = np.random.uniform(sigma_sub_prior[0], sigma_sub_prior[1])
        log_mhm = np.random.uniform(log_mhm_prior[0], log_mhm_prior[1])
        alpha = np.random.uniform(alpha_prior[0], alpha_prior[1])
        delta_los = np.random.uniform(delta_los_prior[0], delta_los_prior[1])
        source_size_1 = np.random.uniform(source_size_prior_1[0], source_size_prior_1[1])
        source_size_2 = np.random.uniform(source_size_prior_2[0], source_size_prior_2[1])
        log10_mhost = np.random.normal(log10_host_mass_prior[0], log10_host_mass_prior[1])
            
        gamma_macro = np.random.uniform(gamma_macro_prior[0], gamma_macro_prior[1])
        params = np.array([sigma_sub, log_mhm, alpha, delta_los, log10_mhost, gamma_macro, source_size_1, source_size_2])
        
        delta_x, delta_y = np.random.normal(0, astrometric_uncertainty), np.random.normal(0, astrometric_uncertainty)
        data = LensedQuasar(x_image + delta_x, y_image + delta_y,  magnifications)
        
        realization = WDMLovell2020(zlens, zsource, 
                                    log_mhm, 
                                    sigma_sub=sigma_sub, 
                                    LOS_normalization=delta_los, 
                                    log_m_host=log10_mhost, 
                                    power_law_index=alpha,
                                   cone_opening_angle_arcsec=cone_opening_angle)
        
        if verbose:
            print('realization contains '+str(len(realization.halos))+ ' halos.')
            print('parameter array: ', params)
        e1_init = np.random.uniform(-0.3, 0.3)
        e2_init = np.random.uniform(-0.3, 0.3)
        g1_init = np.random.uniform(-0.1, 0.1)
        g2_init = np.random.uniform(-0.1, 0.1)
        kwargs_macromodel = [{'theta_E': R_ein_approx, 'center_x': 0., 'center_y': -0.0, 'e1': e1_init, 'e2': e2_init, 
                              'gamma': gamma_macro}, 
               {'gamma1': g1_init, 'gamma2': g2_init}]
        
        main_lens_fit = PowerLawShear(zlens, kwargs_macromodel)
        macromodel = MacroLensModel([main_lens_fit])
        lens_system = QuadLensSystem.shift_background_auto(data, macromodel, zsource, realization)
        
        # this will print the output from the fitting process. Set it to False to save space if you want
        optimizer = HierarchicalOptimization(lens_system)
        kwargs_lens_final, lens_model_full, return_kwargs = optimizer.optimize(data, constrain_params=None,
                                                           param_class_name='free_shear_powerlaw',
                                                                               verbose=verbose)
        
        mags1 = lens_system.quasar_magnification(data.x, 
                                         data.y, source_size_1, lens_model=lens_model_full, 
                                       kwargs_lensmodel=kwargs_lens_final, grid_axis_ratio=0.5, 
                                       grid_resolution_rescale=2., source_model='GAUSSIAN')

        mags2 = lens_system.quasar_magnification(data.x, 
                                         data.y, source_size_2, lens_model=lens_model_full, 
                                       kwargs_lensmodel=kwargs_lens_final, grid_axis_ratio=0.5, 
                                       grid_resolution_rescale=2., source_model='GAUSSIAN')
        
        if parameter_array is None:
            parameter_array = params
        else:
            parameter_array = np.vstack((parameter_array, params))
            
        if magnifications_1 is None:
            magnifications_1 = mags1
            magnifications_2 = mags2
        else:
            magnifications_1 = np.vstack((magnifications_1, mags1))
            magnifications_2 = np.vstack((magnifications_2, mags2))
        print(parameter_array)
        
    filename_parameters = output_path + 'job_'+str(job_index) + '/parameters.txt'
    with open(filename_parameters, 'a') as f:
        nrows, ncols = int(parameter_array.shape[0]), int(parameter_array.shape[1])
        for row in range(0, nrows):
            for col in range(0, ncols):
                f.write(str(np.round(parameter_array[row, col], 6))+' ')
            f.write('\n')
    
    filename_mags1 = output_path + 'job_'+str(job_index) + '/fluxes_1.txt'
    with open(filename_mags1, 'a') as f:
        nrows, ncols = int(magnifications_1.shape[0]), int(magnifications_1.shape[1])
        for row in range(0, nrows):
            for col in range(0, ncols):
                f.write(str(np.round(magnifications_1[row, col], 6))+' ')
            f.write('\n')
            
    filename_mags2 = output_path + 'job_'+str(job_index) + '/fluxes_2.txt'
    with open(filename_mags2, 'a') as f:
        nrows, ncols = int(magnifications_2.shape[0]), int(magnifications_2.shape[1])
        for row in range(0, nrows):
            for col in range(0, ncols):
                f.write(str(np.round(magnifications_2[row, col], 6))+' ')
            f.write('\n')


In [17]:
def inference(samples, model_fluxes, measured_fluxes, flux_uncertainties, n_keep):
    
    """
    samples: an array of samples (combined from all the output files)
    model_fluxes: the modeled image fluxes (combined from all the output files)
    flux_uncertainties: the uncertainties in the flux measurements
    n_keep: the number of samples to keep
    """
    
    perturbed_flux_ratios = np.empty((int(model_fluxes.shape[0]), 3))
    
    for i in range(0, int(model_fluxes.shape[0])):
        delta_f = np.random.normal(0, model_fluxes[i,:] * flux_uncertainties)
        perturbed_fluxes = delta_f + model_fluxes[i,:]
        perturbed_flux_ratios[i, :] = perturbed_fluxes[1:]/perturbed_fluxes[0]
        
    measured_flux_ratios = measured_fluxes[1:] / measured_fluxes[0]
    
    statistic = np.sum(np.sqrt((measured_flux_ratios - perturbed_flux_ratios)**2), axis=1)
    
    inds = np.argsort(statistic)[0:n_keep]
    
    return samples[inds, :], statistic[inds]

[[3 3 3]
 [2 2 2]]


In [4]:
x_image = np.array([-0.347, -0.734, -1.096, 0.207])
y_image = np.array([ 0.964,  0.649, -0.079, -0.148])
magnifications = np.array([[0.88, 1.,  0.474, 0.025]])

output_path = os.getcwd() + '/'
job_index = 1
n_realizations = 2
zlens, zsource = 0.45, 3.69
sigma_sub_prior = [0.0, 0.1]
log_mhm_prior = [4, 10] 
alpha_prior = [-1.95, -1.85]
delta_los_prior = [0.75, 1.25] 
source_size_prior_1 = [1, 5] 
source_size_prior_2 = [40, 80]
log10_host_mass_prior = [13.3, 0.3] #Note that this one is a Gaussian, everything else is uniform prior
gamma_macro_prior = [-1.9, -2.2]

forward_model_wdm(output_path, job_index, n_realizations, zlens, zsource, x_image, y_image, magnifications, 
                  sigma_sub_prior, log_mhm_prior, alpha_prior, delta_los_prior, source_size_prior_1, source_size_prior_2,
                    log10_host_mass_prior, gamma_macro_prior, astrometric_uncertainty=0.003, verbose=True)

reading output to files: 
/Users/danielgilman/Code/quadmodel/notebooks/job_1/parameters.txt
/Users/danielgilman/Code/quadmodel/notebooks/job_1/fluxes_1.txt
/Users/danielgilman/Code/quadmodel/notebooks/job_1/fluxes_2.txt
realization contains 3406 halos.
parameter array:  [ 2.08179573e-02  4.34041820e+00 -1.94217035e+00  1.11036125e+00
  1.34637712e+01 -2.19309282e+00  4.28265342e+00  5.45348262e+01]
optimization 1
aperture size:  100
minimum mass in aperture:  7.0
minimum global mass:  7.0
N foreground halos:  33
N subhalos:  69
10
20
30
40
50
60
70
80
90
100
PSO done... 
source plane chi^2:  77047.5946480774
total chi^2:  77047.5946480774
starting amoeba... 
optimization done.
Recovered source position:  (array([0.21802027, 0.21802025, 0.21802023, 0.21802028]), array([-0.16894562, -0.16894543, -0.16894527, -0.1689454 ]))
optimization 2
aperture size:  0.2
minimum mass in aperture:  0.0
minimum global mass:  7.0
N foreground halos:  40
N subhalos:  91
starting amoeba... 
optimization do